<a href="https://colab.research.google.com/github/cristoffersantoro/gemini-crud-generator/blob/main/Crud_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalando os pacotes necessários

In [ ]:
!pip install -q -U PyGithub
!pip install -q -U google-generativeai
!pip install -q -U firebase-admin

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.4/354.4 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.3/126.3 kB 1.6 MB/s eta 0:00:00


Importando as dependências

In [ ]:
import os
import google.generativeai as genai
import time
import json
from google.colab import drive
import firebase_admin
from firebase_admin import credentials, firestore


Montando o Google Drive

In [ ]:
drive.mount('/content/drive')

Obtendo as credenciais que serão usadas no projeto

In [ ]:
# Exemplo de uso
caminho_arquivo = '/content/drive/MyDrive/tokens.json'  # Substitua pelo caminho correto

with open(caminho_arquivo, 'r') as f:
    dados = json.load(f)

cred = credentials.Certificate('/content/drive/MyDrive/chat-gemini-firebase.json')
git_token = dados['github-token']
generative_ai_key = dados['generative-api-key']
git_user = "cristoffersantoro"
git_repo = "full-stack-fastapi-template"
git_dir_frontend = "frontend/src" # Opcional
git_dir_backend = "backend" # Opcional


Autenticando no Firebase

In [ ]:
firebase_admin.initialize_app(cred)

Autenticando no Generative AI

In [ ]:
genai.configure(api_key=generative_ai_key)

Inicializando o Modelo do Generative AI

In [ ]:
# Set up the model
generation_config = {
  "temperature": 0.0
}

model = genai.GenerativeModel(model_name="gemini-1.0-pro",
                              generation_config=generation_config)

Métodos para listagem de arquivos no Github

In [ ]:
from github import Github

def listar_e_ler_arquivos_recursivamente(token, usuario, repositorio, caminho_diretorio=""):
  """
  Lista recursivamente os arquivos em um repositório do GitHub e lê seu conteúdo.

  Args:
    token: O token de acesso pessoal do GitHub.
    usuario: O nome de usuário do proprietário do repositório.
    repositorio: O nome do repositório.
    caminho_diretorio: O caminho para o diretório (opcional).

  Returns:
    Um dicionário onde as chaves são os caminhos dos arquivos e os valores são os conteúdos dos arquivos.
  """
  # Criar um objeto Github com o token
  g = Github(token)

  # Obter o repositório
  repo = g.get_repo(f"{usuario}/{repositorio}")

  # Obter o conteúdo do diretório
  conteudo = repo.get_contents(caminho_diretorio)

  # Listar arquivos e ler conteúdo recursivamente
  arquivos_conteudo = {}
  while conteudo:
    arquivo_conteudo = conteudo.pop(0)
    if arquivo_conteudo.type == "dir":
      conteudo.extend(repo.get_contents(arquivo_conteudo.path))
    else:
      # Obter conteúdo do arquivo
      conteudo_bruto = arquivo_conteudo.decoded_content
      conteudo_texto = conteudo_bruto.decode("utf-8", errors="ignore")
      arquivos_conteudo[arquivo_conteudo.path] = conteudo_texto

  return arquivos_conteudo

def listar_e_ler_arquivo(token, usuario, repositorio, arquivos_conteudo, caminho_arquivo):
  """
  Lista recursivamente os arquivos em um repositório do GitHub e lê seu conteúdo.

  Args:
    token: O token de acesso pessoal do GitHub.
    usuario: O nome de usuário do proprietário do repositório.
    repositorio: O nome do repositório.
    arquivos_conteudo: Um dicionário onde as chaves são os caminhos dos arquivos e os valores são os conteúdos dos arquivos.
    caminho_arquivo: O caminho para o arquivo.

  Returns:
    Um dicionário onde as chaves são os caminhos dos arquivos e os valores são os conteúdos dos arquivos.
  """
  # Criar um objeto Github com o token
  g = Github(token)

  # Obter o repositório
  repo = g.get_repo(f"{usuario}/{repositorio}")

  # Obter o conteúdo do diretório
  conteudo = repo.get_contents(caminho_arquivo)

  conteudo_bruto = conteudo.decoded_content
  conteudo_texto = conteudo_bruto.decode("utf-8", errors="ignore")
  arquivos_conteudo[conteudo.path] = conteudo_texto

  return arquivos_conteudo




Lista conteúdo arquivos Frontend do repositório Github

In [ ]:

arquivos_conteudo_frontend = listar_e_ler_arquivos_recursivamente(git_token, git_user, git_repo, git_dir_frontend)
arquivos_conteudo_frontend = listar_e_ler_arquivo(git_token, git_user, git_repo, arquivos_conteudo_frontend, "frontend/README.md")


Inicialização do Database do Firebase

In [ ]:
db = firestore.client()

Salva os dados obtidos do Github no firebase. Com isso evita a necessidade de ficar usando a API do Github que possui limites de requisição

In [ ]:

usuarios_ref = db.collection('model')

for caminho, conteudo in arquivos_conteudo_frontend.items():
  dados = {
    'file': {
        'path': caminho,
        'content': conteudo
    }
  }

  usuarios_ref.add(dados)



Lista os arquivos salvos no Firebase que irão compor a mensagem enviada ao Chat do Gemini AI

In [ ]:
# Referência à collection
usuarios_ref = db.collection('model')

# Obtendo todos os documentos
docs = usuarios_ref.stream()

for doc in docs:
    print(f'{doc.id} => {doc.to_dict()}')

Preparando o modelo de chat para receber os inputs

In [ ]:
chat = model.start_chat(history=[])

Envia o input introdutório

Trecho para enviar mensagem via Chat para o Gemini AI com retry de 3 tentativas para o caso de falha.

In [ ]:
max_tentativas = 3

In [ ]:
def enviar_mensagem_chat(mensagem, tentativa):
  if tentativa > max_tentativas:
    return
  time.sleep(5)
  try:
    chat.send_message(mensagem)
  except:
    print(f"tentativa {tentativa} falhou. Tentando novamente...")
    enviar_mensagem_chat(mensagem, tentativa+1)

Core da aplicação. Este ponto é responsável por orquestrar todo o conteúdo para o Gemini AI e usando o few-shots, irá treinar o modelo para responder
aquilo que preciso.

In [ ]:
# Input introdutório
chat.send_message("Analise os arquivos contidos nesse template react com chackra-ui. Enviarei cada arquivo separadamente para analise")

# Referência à collection
usuarios_ref = db.collection('model')

# Obtendo todos os documentos
docs = usuarios_ref.stream()

for doc in docs:
  doc_dict = doc.to_dict()
  print(f"enviando o arquivo: {doc_dict['file']['path']}")
  enviar_mensagem_chat(f"filepath: {doc_dict['file']['path']}, content: {doc_dict['file']['content']}", 1)

response = chat.send_message("""
input: analise o content do filepath frontend/README.MD"
input: analise a estrutura do content do filepath frontend/src/routes/_layout/items.tsx
input: Com base na estrurura passada, explique para mim como foi
construída a page de items, explicando cada referência, onde as rotas foram adicionadas, quais componentes e models foram criados,
quais propriedades existem""")


Por conta de erros de timeout na comunicação com o Gemini AI, optei por registrar o histórico de chat no banco de dados, para posteriormente continuar
sem precisar fazer todo um reprocessamento.

In [ ]:
history_ref = db.collection('chat-history')
history_ref.add({'history': str(chat.history)})


(DatetimeWithNanoseconds(2024, 5, 11, 15, 32, 58, 699702, tzinfo=datetime.timezone.utc),
 <google.cloud.firestore_v1.document.DocumentReference at 0x7d78d462a620>)

Responsável por resgatar o histórico salvo

In [ ]:
# Referência à collection
history_ref = db.collection('chat-history')

query = history_ref.limit(1)
history_docs = query.stream()

for history_doc in history_docs:
  string_lista = history_doc.to_dict()['history']
  string_json = json.dumps(string_lista)
  lista_convertida = json.loads(string_json)

chat.history = lista_convertida


Prompt final, onde com base no que foi treinado acima, deverá retornar o output esperado. Precisa ser refinado.

In [ ]:
chat.send_message("""
  Agora baseado em toda a análise feita, gostaria que gerasse para mim uma nova tela com o nome de Produtos,
  que contém os campos Name, Description e Quantity, e seguindo o mesmo modelo da tela de Items, gere para mim todas
  as telas e componentes necessários para nova tela, exemplo: AddProduct, EditProdut, _Layout, etc, e atualize os arquivos existentes
  no projeto que são necessários, como routes, components, etc
                  """)